In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
import matplotlib.pyplot as plt
from evallm.experiments.sequence_completion_summary import (
    sequence_completion_results,
    display_prompt as display_prompt_sc,
)

from evallm.experiments.transducer_summary import *
from evallm.experiments.transducer_experiment import *
import evallm

In [3]:
num_dfas = 10

In [4]:
res_4os = run_transducer_experiment(
    model="gpt-4o-2024-05-13",
    num_repeats_per_dfa=30,
    sample_dfa_spec=sample_dfa_spec,
    prompter=prompt_by_key["Basic"]["chat"],
    num_dfas=num_dfas,
)

In [5]:
res_o1s = run_transducer_experiment(
    # seed=3,
    model="o1-preview-2024-09-12",
    num_repeats_per_dfa=30,
    sample_dfa_spec=sample_dfa_spec,
    prompter=prompt_by_key["Basic"]["chat"],
    # print_completions=True,
    num_dfas=num_dfas,
)

In [6]:
def compute_frac(x):
    numer = (np.array(x.success_rate_each) == 1).sum()
    denom = (np.array(x.success_rate_each) != 0.5).sum()
    return numer, denom


def render_frac(x):
    numer, denom = compute_frac(x)
    return f"{numer:.0f}/{denom:.0f}"

In [7]:
table = pd.DataFrame(
    {
        "o1-preview": [render_frac(x) for x in res_o1s],
        "gpt-4o": [render_frac(x) for x in res_4os],
        r"6-\textsc{Gram}": [
            f"{x.kgram_success_rates_each[6 - 2] * 30:.0f}/30" for x in res_4os
        ],
    }
)

In [8]:
table

,o1-preview,gpt-4o,6-\textsc{Gram}
0,25/30,27/30,26/30
1,23/29,24/30,25/30
2,19/30,23/30,28/30
3,22/30,23/30,28/30
4,29/29,30/30,30/30
5,19/30,24/30,30/30
6,17/29,23/30,25/30
7,23/30,25/30,26/30
8,21/30,28/30,30/30
9,29/30,29/30,30/30


In [9]:
text = ""
text += r"\begin{tabular}{|r|c|c|c|}" + "\n"
text += r"\hline" + "\n"
text += "&".join(["DFA", *table.columns])
text += "\\\\\n"
for i, row in table.iterrows():
    text += r"\hline" + "\n"
    text += "&".join([str(i + 1), *row])
    text += "\\\\\n"
text += r"\hline" + "\n"
text += r"\end{tabular}"

In [10]:
print(text)

\begin{tabular}{|r|c|c|c|}
\hline
DFA&o1-preview&gpt-4o&6-\textsc{Gram}\\
\hline
1&25/30&27/30&26/30\\
\hline
2&23/29&24/30&25/30\\
\hline
3&19/30&23/30&28/30\\
\hline
4&22/30&23/30&28/30\\
\hline
5&29/29&30/30&30/30\\
\hline
6&19/30&24/30&30/30\\
\hline
7&17/29&23/30&25/30\\
\hline
8&23/30&25/30&26/30\\
\hline
9&21/30&28/30&30/30\\
\hline
10&29/30&29/30&30/30\\
\hline
\end{tabular}


In [11]:
np.array([compute_frac(x) for x in res_o1s]).sum(0)

array([227, 297])

In [12]:
np.array([compute_frac(x) for x in res_4os]).sum(0)

array([256, 300])